In [43]:
import sys
import os
import dspy
import boto3
from langchain_aws import BedrockLLM as Bedrock

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [44]:
try: # When on google Colab, let's clone the notebook so we download the cache.
    repo_path = 'dspy'
    # !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

In [46]:
from dspy.retrieve.chromadb_rm import ChromadbRM
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# embedding_function = OpenAIEmbeddingFunction(
#     api_key=os.environ.get('OPENAI_API_KEY'),
#     model_name="text-embedding-ada-002"
# )
embedding_function = SentenceTransformerEmbeddingFunction()

retriever_model = ChromadbRM(
    'JetBlueHelp_stef',
    embedding_function=embedding_function,
    persist_directory='/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/chromadb2/',
    k=5
)

question = "Can I take holy water on the plane? if yes, what is the permitted quantity?"
retriever_model(question, k=1)

[{'id': '201',
  'score': 0.8914483785629272,
  'long_text': '7/28/24, 23:08Holy Water / Zamzam Water | JetBlue\nPage 2 of 3https://www.jetblue.com/help/holy-waterNeed help?Holy Water & Zamzam Water on JetBlueFlightsJetBlue will accept one jerry can containing up to 10 liters (2.64 gallons) of Holywater or Zamzam water as a checked bag\xa0at no extra charge.The jerry can must be properly packed in a plastic covering to avoid leakageand damage to other bags.Jerry cans containing Holy or Zamzam water are\xa0not permitted as a carry-onbag.If more than one jerry can is checked, the extra jerry cans will be subject tostandard excess bag\xa0charges.Was this page helpful?YesNo\nSearch for answers',
  'metadatas': {'page': 1,
   'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Holy Water : Zamzam Water | JetBlue.pdf'}}]

In [47]:
use_model = 'llama3'
if use_model == 'cohere':
    llm_model = dspy.Cohere(model='command-xlarge-nightly', api_key=os.getenv("COHERE_API_KEY"))
if use_model == 'phi':
    llm_model = dspy.OllamaLocal(model='phi')
else: # use phi local model
    llm_model = dspy.OllamaLocal(model='llama3')

print(f"Using model: {use_model}")

Using model: llama3


In [48]:
dspy.settings.configure(lm=llm_model, rm=retriever_model) # configure dspy

In [10]:
class BasicQA(dspy.Signature):
    # """Answer questions with short factoid answers."""
    """Act as a ChatBot on JetBlue airlines website, answer queries by customers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="No more than 20 words. Use metrics")

In [11]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=question)

# Print the input and the prediction.
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

Question: Can I take holy water on the plane? if yes, what is the permitted quantity?
Predicted Answer: Question: Can I take holy water on the plane? if yes, what is the permitted quantity?

Answer: Yes, you can bring holy water in a 1-quart (32 oz) container.


In [12]:
llm_model.inspect_history(n=1)




Act as a ChatBot on JetBlue airlines website, answer queries by customers.

---

Follow the following format.

Question: ${question}
Answer: No more than 20 words. Use metrics

---

Question: Can I take holy water on the plane? if yes, what is the permitted quantity?
Answer: Question: Can I take holy water on the plane? if yes, what is the permitted quantity?

Answer: Yes, you can bring holy water in a 1-quart (32 oz) container.





'\n\n\nAct as a ChatBot on JetBlue airlines website, answer queries by customers.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: No more than 20 words. Use metrics\n\n---\n\nQuestion: Can I take holy water on the plane? if yes, what is the permitted quantity?\nAnswer:\x1b Question: Can I take holy water on the plane? if yes, what is the permitted quantity?\n\nAnswer: Yes, you can bring holy water in a 1-quart (32 oz) container.\x1b\n\n\n'

In [13]:
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Question: Can I take holy water on the plane? if yes, what is the permitted quantity?
Thought: We need to check JetBlue's policies and regulations regarding liquids, gels, and aerosols.
Predicted Answer: Yes, you can bring holy water on board! The Transportation Security Administration (TSA) allows 3.4 ounces or less of liquids, gels, and aerosols in your carry-on bag.


In [14]:
retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(question).passages

print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

Top 3 passages for question: Can I take holy water on the plane? if yes, what is the permitted quantity? 
 ------------------------------ 

1] 7/28/24, 23:08Holy Water / Zamzam Water | JetBlue
Page 2 of 3https://www.jetblue.com/help/holy-waterNeed help?Holy Water & Zamzam Water on JetBlueFlightsJetBlue will accept one jerry can containing up to 10 liters (2.64 gallons) of Holywater or Zamzam water as a checked bag at no extra charge.The jerry can must be properly packed in a plastic covering to avoid leakageand damage to other bags.Jerry cans containing Holy or Zamzam water are not permitted as a carry-onbag.If more than one jerry can is checked, the extra jerry cans will be subject tostandard excess bag charges.Was this page helpful?YesNo
Search for answers 

2] 7/28/24, 23:08Holy Water / Zamzam Water | JetBlue
Page 1 of 3https://www.jetblue.com/help/holy-waterHoly Water / Zamzam WaterWe are proud to support your religions and traditions while you travel with us.Search for answers Hel

In [15]:
class GenerateAnswer(dspy.Signature):
    """Act as a ChatBot on JetBlue airlines website, answer queries by customers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="no more than 20 words")

In [16]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [17]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

In [18]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

100%|██████████| 20/20 [04:04<00:00, 12.20s/it]

Bootstrapped 0 full traces after 20 examples in round 0.


In [19]:
# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(question)

# Print the contexts and the answer.
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Can I take holy water on the plane? if yes, what is the permitted quantity?
Predicted Answer: Yes, you can take holy water on the plane, and the permitted quantity is up to 10 liters (2.64 gallons).
Retrieved Contexts (truncated): ['7/28/24, 23:08Holy Water / Zamzam Water | JetBlue\nPage 2 of 3https://www.jetblue.com/help/holy-waterNeed help?Holy Water & Zamzam Water on JetBlueFlightsJetBlue will accept one jerry can containing up...', '7/28/24, 23:08Holy Water / Zamzam Water | JetBlue\nPage 1 of 3https://www.jetblue.com/help/holy-waterHoly Water / Zamzam WaterWe are proud to support your religions and traditions while you travel with...', '7/28/24, 23:08Holy Water / Zamzam Water | JetBlue\nPage 3 of 3https://www.jetblue.com/help/holy-waterGet To Know UsPoliciesJetBlue In ActionStay ConnectedJoin our email listDownload the JetBlue mobile ...']


In [21]:
compiled_rag.save("compiled_models/compiled_RAG.json")

compiled_rag_saved = RAG()
compiled_rag_saved.load("compiled_models/compiled_RAG.json")

[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x1432495a0>), ('generate_answer', Predict(StringSignature(context, question -> rationale, answer
    instructions='Act as a ChatBot on JetBlue airlines website, answer queries by customers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'no more than 20 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]


In [23]:
questions = [
    "Can I take holy water on the plane? if yes, what is the permitted quantity?",
    "what is the check-in baggage weight limit?",
    "How to do a online baggage claim report?",
]

In [26]:
for ques in questions:
    pred = compiled_rag_saved(ques)

    # Print the contexts and the answer.
    print(f"Human: {ques}")
    print(f"AI: {pred.answer}\n")

Human: Can I take holy water on the plane? if yes, what is the permitted quantity?
AI: Yes, you can take holy water on the plane, and the permitted quantity is up to 10 liters (2.64 gallons).

Human: what is the check-in baggage weight limit?
AI: Hello there! I'm happy to help you with your question. According to JetBlue's checked bag policy, the maximum weight limit for a checked bag is 50 pounds (22.68 kg). If your bag exceeds this weight limit, it will be considered overweight and you'll need to pay an excess bag fee. Let me know if you have any other questions or concerns!

Human: How to do a online baggage claim report?
AI: I'd be happy to help!

According to JetBlue's website, it seems that you can add bags and pay fees at any time leading up to your departure. You can also indicate how many bags you're bringing and pay the applicable fees during the check-in process within 24 hours of your flight's departure.

To report a baggage claim online, you can follow these steps:

1. Go 

In [30]:
st_chat_history ="""Human: what is the check-in baggage weight limit?
AI: Hello there! I'm happy to help you with your question. According to JetBlue's checked bag policy, the maximum weight limit for a checked bag is 50 pounds (22.68 kg). If your bag exceeds this weight limit, it will be considered overweight and you'll need to pay an excess bag fee. Let me know if you have any other questions or concerns!

Human: Can I take holy water on the plane? if yes, what is the permitted quantity?
AI: Yes, you can take holy water on the plane, and the permitted quantity is up to 10 liters (2.64 gallons).

Human: How to do a online baggage claim report?
AI: I'd be happy to help!

According to JetBlue's website, it seems that you can add bags and pay fees at any time leading up to your departure. You can also indicate how many bags you're bringing and pay the applicable fees during the check-in process within 24 hours of your flight's departure.

To report a baggage claim online, you can follow these steps:

1. Go to JetBlue's website and log in to your account.
2. Click on "Manage Your Trip" and select the trip for which you want to file a baggage claim.
3. Scroll down to the "Baggage" section and click on "File a Baggage Claim".
4. Fill out the required information, including your flight details, bag type"""


In [39]:
class Chatbot(dspy.Signature):
    # """Answer questions with short factoid answers."""
    """Act as a ChatBot on JetBlue airlines website, answer queries by customers based on facts and context."""

    context = dspy.InputField(desc="may contain relevant facts")
    chat_history = dspy.InputField(desc="contains the history of the chat so far")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="No more than 20 words. may have metrics")

class RAG_chatbot(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(Chatbot)
        # self.chat_history = st_chat_history
    
    def get_chat_history(self, st_chat_history):
        chat_history = "Chat History : \n"
        for chat_history_item in st_chat_history:
            chat_history += f"{chat_history_item[0]} : {chat_history_item[1]} \n"
        
        return chat_history
    
    def forward(self, question):
        context = self.retrieve(question).passages
        # chat_history = self.get_chat_history(st_chat_history)
        prediction = self.generate_answer(context=context, question=question, chat_history=st_chat_history)

        return dspy.Prediction(context=context, answer=prediction.answer)

In [40]:
for item in trainset:
    print(item)

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'})
Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'})
Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'})
Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'})
Example({'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'answer': '1874'}) (input_keys={'question'})
Example({'question': 'Which is taller, the Empire State B

In [41]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG_chatbot(), trainset=trainset)

100%|██████████| 20/20 [03:50<00:00, 11.53s/it]

Bootstrapped 0 full traces after 20 examples in round 0.


In [42]:
compiled_rag.save("compiled_models/chatbot_RAG.json")

compiled_rag_saved = RAG()
compiled_rag_saved.load("compiled_models/chatbot_RAG.json")

[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x143d34070>), ('generate_answer', Predict(StringSignature(context, chat_history, question -> rationale, answer
    instructions='Act as a ChatBot on JetBlue airlines website, answer queries by customers based on facts and context.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    chat_history = Field(annotation=str required=True json_schema_extra={'desc': 'contains the history of the chat so far', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(

In [ ]:
# create a training example set
# dspy.Example(question="", answer="", chat_history="")